# Generate Friction Surface

In [2]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph
#import GOSTnets.load_osm as osm

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.insert(0, "/home/jupyter-wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.osmMisc as osmMisc
from GOSTRocks.misc import tPrint

sys.path.append("../src")
import GOSTNetsRaster.market_access as ma
import GOSTNetsRaster.conversion_tables as speed_tables

In [31]:
# define input data
base_folder = '/home/public/Projects/PAK_GOSTNets/Data/'
aoi_file = '/home/public/Projects/PAK_GOSTNets/Data/geoBoundaries-PAK-ADM1.geojson'
global_landcover = '/home/public/GLOBAL/Landcover/Copernicus/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif'
osm_file = '/home/public/Projects/PAK_GOSTNets/Data/pakistan-latest.osm.pbf'
sel_province = "Sindh"

output_folder = os.path.join(base_folder, sel_province)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
local_lc_file = os.path.join(output_folder, "LANDCOVER.tif")
local_osm = os.path.join(output_folder, "OSM_highways.osm.pbf")
local_friction = os.path.join(output_folder, "friction_surface.tif")

In [4]:
inA = gpd.read_file(aoi_file)
inA = inA.loc[inA['shapeName'] == sel_province]

In [5]:
if not os.path.exists(local_lc_file):
    inR = rasterio.open(global_landcover)
    rMisc.clipRaster(inR, inA, local_lc_file)
inR = rasterio.open(local_lc_file)

In [6]:
if not os.path.exists(local_osm):
    osm_extract = osmMisc.osmExtraction(osmosisCmd = "/home/jupyter-wb411133/Code/osmosis/bin/osmosis",tempFile = "/home/jupyter-wb411133/Code/tempFile.bat")
    osm_extract.extractHighways(osm_file, local_osm, bounds = inA.total_bounds, execute=True)

# Generate friction surface

In [ ]:
roads = osmMisc.convertOSMPBF_DataFrame(local_osm, "lines")
roads['speed'] = roads['highway'].map(speed_tables.osm_speed_dict).fillna(10)

In [83]:
importlib.reload(ma)
ma.generate_roads_lc_friction?

Signature:
ma.generate_roads_lc_friction(
    lc_file,
    sel_roads,
    lc_travel_table=None,
    min_lc_val=0.01,
    min_road_speed=0.01,
    speed_col='speed',
    resolution=100,
    out_file='',
)
Docstring:
Combine a landcover dataset and a road network dataset to create
a friction surface. See generate_lc_friction and generate_road_friction for more details

Returns
dictionary of 'meta'[dictionary] and 'friction'[numpy array] - meta is metadata required to write rasterio
    friction is the resulting friction surface used to run in GOSTNets Raster
File:      ~/Code/GOSTNets_Raster/src/GOSTNetsRaster/market_access.py
Type:      function


In [82]:
res = ma.generate_roads_lc_friction(inR, roads)

/home/jupyter-wb411133/Code/GOSTNets_Raster/notebooks/../src/GOSTNetsRaster/market_access.py:150: RuntimeWarning: divide by zero encountered in true_divide
  res = resolution / (res * 1000 / (60 * 60)) # km/h --> m/s * resolution of image in metres
